In [1]:
import PyPDF2
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from transcript_functions import *

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rawin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rawin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
filename = '2019_proxy.pdf' 

pdfFileObj = open(filename,'rb')

pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

text = []
for i in range(1, pdfReader.numPages):
    pageObj = pdfReader.getPage(i)
    page = pageObj.extractText()
    page = page.replace('\n \n',', ')
    page = page.replace('\n','')
    page = page.replace('\d+', '')
    text.append(page)

In [3]:
len(text)

110

In [4]:
df = pd.DataFrame(text, columns=['raw_text'])

In [5]:
#tokenize
df["tokens"] = df["raw_text"].map(nltk.word_tokenize)

#make lowercase
df["tokens_clean"] = df["tokens"].map(lambda x: [word.lower() for word in x])

#remove stopwords
stop_words = set(stopwords.words('english'))
df["tokens_clean"] = df["tokens_clean"].map(lambda x: [word for word in x if word not in stop_words])

#remove punctuation
punctuation = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~,."
df["tokens_clean"] = df["tokens_clean"].map(lambda x: [word for word in x if word not in punctuation])

Lemmatize with POS tagging:

In [6]:
# tag Part of Speech
df["tokens_pos"] = df["tokens_clean"].map(nltk.pos_tag)
# filter words with pos not in pos_keep
pos_keep = ["NN", "NNS","VB", "VBD", "VBN", "VBP", "VBZ"] #take out "JJ", "JJR", "JJS"
df["tokens_pos"] = df["tokens_pos"].map(lambda x: [tup for tup in x if tup[1] in pos_keep])

# lemmatize with POS
lemmatizer = WordNetLemmatizer()
df["tokens_pos"] = df["tokens_pos"].map(lambda x: [lemmatizer.lemmatize(tup[0], get_wordnet_pos(tup[1])) for tup in x ])

df["corpus"] = df["tokens_pos"].map(lambda x: ' '.join(x))

#remove numbers
df['corpus'] = df['corpus'].str.replace('\d+', '')

#tokenize
df["tokens_final"] = df["corpus"].map(nltk.word_tokenize)

Lemmatize without POS tagging:

In [7]:
# def lemmatize_text(text):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(w) for w in text] 

# df["tokens_clean"] = df["tokens_clean"].apply(lemmatize_text)

# df["corpus"] = df["tokens_clean"].map(lambda x: ' '.join(x))

# #remove numbers
# df['corpus'] = df['corpus'].str.replace('\d+', '')

# #tokenize
# df["tokens_final"] = df["corpus"].map(nltk.word_tokenize)

Further Pre-processing

In [8]:
# remove_words = ["director","meeting","committee","mr.","proxy","unit","total","plan","u.s.",
#                 "ceo","walton","stock","vote","walmar˜"]

# df["tokens_final"] = df["tokens_final"].map(lambda x: [word for word in x if word.lower() not in remove_words])

In [9]:
df.head()

,raw_text,tokens,tokens_clean,tokens_pos,corpus,tokens_final
0,At Walmar˜ we save people money so they can li...,"[At, Walmar˜, we, save, people, money, so, the...","[walmar˜, save, people, money, live, better.th...","[walmar˜, save, people, money, live, better.th...",walmar˜ save people money live better.the defi...,"[walmar˜, save, people, money, live, better.th..."
1,3Messages from our Chairman and our Lead Indep...,"[3Messages, from, our, Chairman, and, our, Lea...","[3messages, chairman, lead, independent, direc...","[3messages, chairman, lead, directorwe, please...",messages chairman lead directorwe please atten...,"[messages, chairman, lead, directorwe, please,..."
2,4Fiscal 2019 HighlightsStrategy and Per˚ormanc...,"[4Fiscal, 2019, HighlightsStrategy, and, Per˚o...","[4fiscal, 2019, highlightsstrategy, per˚ormanc...","[highlightsstrategy, per˚ormancewalmar˜, deliv...",highlightsstrategy per˚ormancewalmar˜ deliver ...,"[highlightsstrategy, per˚ormancewalmar˜, deliv..."
3,5$514.4 billionFiscal 2019 Total Revenues$16 b...,"[5, $, 514.4, billionFiscal, 2019, Total, Reve...","[5, 514.4, billionfiscal, 2019, total, revenue...","[total, revenue, billioninvestment, flipkar˜, ...",total revenue billioninvestment flipkar˜ ecomm...,"[total, revenue, billioninvestment, flipkar˜, ..."
4,6Notice of 2019 Annual Shareholders' MeetingHo...,"[6Notice, of, 2019, Annual, Shareholders, ', M...","[6notice, 2019, annual, shareholders, meetingh...","[shareholder, meetinghow, shareholders™, meeti...",shareholder meetinghow shareholders™ meeting˛w...,"[shareholder, meetinghow, shareholders™, meeti..."


In [10]:
# count_vectorizer = CountVectorizer()
# count_data = count_vectorizer.fit_transform(df["corpus"])

In [11]:
# number_topics = 15
# number_words = 10

# # fix random_state at 33
# lda = LDA(n_components=number_topics, n_jobs=-1, random_state =33)
# lda.fit(count_data)
       
# print_topics(lda, count_vectorizer, number_words)

Topic 0: Stock value  
Topics 2, 3, 6: Board/Management  
Topic 8: ecommerce  
Topic 10:  
Topic 11: Performance incentives  
Topic 14: Compensation

In [12]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [13]:
dictionary = gensim.corpora.Dictionary(df["tokens_final"])

dictionary.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)

In [14]:
bow_corpus = [dictionary.doc2bow(word) for word in df["tokens_final"]]

In [15]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=15, id2word=dictionary, 
                                       passes=2, workers=2, chunksize=100, random_state=1000)

In [16]:
pp.pprint(lda_model.print_topics())

[   (   0,
        '0.046*"sale" + 0.029*"currency" + 0.025*"calculate" + 0.022*"income" '
        '+ 0.021*"item" + 0.020*"plan" + 0.015*"measure" + 0.015*"incentive" + '
        '0.014*"exclude" + 0.014*"purpose"'),
    (   1,
        '0.039*"director" + 0.016*"shareholder" + 0.013*"committee" + '
        '0.012*"governance" + 0.012*"walmar˜" + 0.009*"receive" + '
        '0.009*"audit" + 0.009*"mr." + 0.009*"statement" + '
        '0.009*"relationship"'),
    (   2,
        '0.051*"audit" + 0.035*"committee" + 0.020*"accountant" + '
        '0.020*"service" + 0.017*"company™s" + 0.015*"review" + 0.013*"ey" + '
        '0.012*"statement" + 0.011*"shareholder" + 0.011*"policy"'),
    (   3,
        '0.024*"stock" + 0.018*"plan" + 0.015*"unit" + 0.015*"per˜ormance" + '
        '0.013*"award" + 0.012*"grant" + 0.011*"equity" + 0.011*"amount" + '
        '0.011*"value" + 0.010*"associate"'),
    (   4,
        '0.030*"director" + 0.017*"meeting" + 0.016*"member" + '
        '0.014*"busin

In [17]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

In [18]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.075049 -0.003079       1        1  14.039927
4     -0.075143  0.020841       2        1  11.860739
6     -0.017937  0.104628       3        1  11.803324
11    -0.070862 -0.209906       4        1  10.329453
9      0.053105 -0.045028       5        1   7.860330
0      0.201981 -0.025584       6        1   7.602670
7     -0.070390  0.070478       7        1   6.120590
12     0.054519  0.046129       8        1   6.077286
2     -0.058033  0.010302       9        1   6.076448
1     -0.095149 -0.010355      10        1   4.846640
5      0.019788  0.016873      11        1   4.626002
14     0.031684  0.018610      12        1   2.954167
10    -0.064075  0.043599      13        1   2.474823
13     0.000251 -0.057315      14        1   2.368062
8      0.015214  0.019809      15        1   0.959541, topic_info=          Term        Freq       Total Category  logprob  loglift
115   director  258.000000  258.000000  Default  30.0000  30.0000
435      audit  116.000000  116.000000  Default  29.0000  29.0000
333       sale   99.000000   99.000000  Default  28.0000  28.0000
429       vote  120.000000  120.000000  Default  27.0000  27.0000
102  committee  184.000000  184.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
354          ł    0.612564   46.054253  Topic15  -5.6618   0.3265
169    meeting    0.686133  176.869492  Topic15  -5.5484  -0.9056
115   director    0.703749  258.337402  Topic15  -5.5231  -1.2591
81       award    0.616881   66.634422  Topic15  -5.6548  -0.0358
673    service    0.597791   59.117294  Topic15  -5.6862   0.0524

[1033 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
65        1  0.073113            .
65        2  0.219340            .
65        3  0.146227            .
65        4  0.048742            .
65        5  0.121855            .
...     ...       ...          ...
1484      1  0.233448     ﬁcovered
1484      7  0.466896     ﬁcovered
2832      5  0.395825  ﬁinvestorsﬂ
2909     12  0.633758   ﬁrepor˜edﬂ
2773      4  0.571850      ﬁstreet

[2750 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 7, 12, 10, 1, 8, 13, 3, 2, 6, 15, 11, 14, 9])